In [11]:
import pandas as pd

# Charger les données
df = pd.read_csv("full_grouped.csv")  

# Renommer les colonnes pour simplifier
df = df.rename(columns={
    'Date': 'Date',
    'Country/Region': 'Country',
    'Confirmed': 'Confirmed',
    'Deaths': 'Deaths',
    'Recovered': 'Recovered',
    'New cases': 'New cases',
    'New deaths': 'New deaths',
    'New recovered': 'New recovered'
})

# Convertir la date et créer une colonne mois
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()

# Garder uniquement les colonnes nécessaires
df = df[['Month', 'Country', 'Confirmed', 'Deaths', 'Recovered',
         'New cases', 'New deaths', 'New recovered']]

# Convertir les colonnes temporelles en valeurs absolues
for col in ['New cases', 'New deaths', 'New recovered']:
    df[col] = df[col].abs()

# Grouper par mois et pays
df_grouped = df.groupby(['Month', 'Country'], as_index=False).agg({
    'Confirmed': 'last',
    'Deaths': 'last',
    'Recovered': 'last',
    'New cases': 'sum',
    'New deaths': 'sum',
    'New recovered': 'sum'
})

# Ajouter une colonne Id auto-incrémentée
df_grouped.insert(0, 'Id', range(1, len(df_grouped) + 1))

# Ajouter une colonne 'nom' avec la valeur 'COVID-19' au début du DataFrame
df_grouped.insert(1, 'Name', 'COVID-19')

# Vérifier la cohérence des données
if df.isnull().any().any():
    print("Les données contiennent des valeurs nulles. Vérifiez le fichier source.")

# Vérifier que les colonnes numériques ne contiennent pas de valeurs négatives
for col in ['Confirmed', 'Deaths', 'Recovered', 'New cases', 'New deaths', 'New recovered']:
    if (df[col] < 0).any():
        print(f"La colonne '{col}' contient des valeurs négatives.")

# Vérifier que les colonnes nécessaires sont présentes
required_columns = ['Month', 'Country', 'Confirmed', 'Deaths', 'Recovered', 'New cases', 'New deaths', 'New recovered']
for col in required_columns:
    if col not in df.columns:
        print(f"La colonne requise '{col}' est manquante dans les données.")

MARGIN = 50
inconsistencies = []

for country, group in df_grouped.groupby('Country'):
    group = group.sort_values('Month').reset_index(drop=True)
    for i in range(1, len(group)):
        current = group.iloc[i]
        prev = group.iloc[i - 1]

        # Vérification décès
        expected_deaths = prev['Deaths'] + current['New deaths']
        if abs(current['Deaths'] - expected_deaths) > MARGIN:
            print(f"⚠️ Incohérence pour '{country}' au mois de {current['Month']} [Deaths]: "
                  f"{current['Deaths']} ≠ {prev['Deaths']} + {current['New deaths']} (tolérance ±{MARGIN})")
            inconsistencies.append({
                'Country': country,
                'Month': current['Month'],
                'Field': 'Deaths',
                'Actual': current['Deaths'],
                'Expected': expected_deaths
            })

        # Vérification cas confirmés
        expected_confirmed = prev['Confirmed'] + current['New cases']
        if abs(current['Confirmed'] - expected_confirmed) > MARGIN:
            print(f"⚠️ Incohérence pour '{country}' au mois de {current['Month']} [Confirmed]: "
                  f"{current['Confirmed']} ≠ {prev['Confirmed']} + {current['New cases']} (tolérance ±{MARGIN})")
            inconsistencies.append({
                'Country': country,
                'Month': current['Month'],
                'Field': 'Confirmed',
                'Actual': current['Confirmed'],
                'Expected': expected_confirmed
            })

        # Vérification récupérations
        expected_recovered = prev['Recovered'] + current['New recovered']
        if abs(current['Recovered'] - expected_recovered) > MARGIN:
            print(f"⚠️ Incohérence pour '{country}' au mois de {current['Month']} [Recovered]: "
                  f"{current['Recovered']} ≠ {prev['Recovered']} + {current['New recovered']} (tolérance ±{MARGIN})")
            inconsistencies.append({
                'Country': country,
                'Month': current['Month'],
                'Field': 'Recovered',
                'Actual': current['Recovered'],
                'Expected': expected_recovered
            })


inconsistencies_df = pd.DataFrame(inconsistencies)
inconsistencies_df.head()

# Sauvegarder le résultat dans le répertoire courant
df_grouped.to_csv("data_etl_output.csv", index=False)

print("\nETL terminé. Fichier sauvegardé sous : data_etl_output.csv")

⚠️ Incohérence pour 'Algeria' au mois de 2020-03-01 00:00:00 [Recovered]: 46 ≠ 0 + 200 (tolérance ±50)
⚠️ Incohérence pour 'Algeria' au mois de 2020-05-01 00:00:00 [Recovered]: 5748 ≠ 1779 + 4043 (tolérance ±50)
⚠️ Incohérence pour 'Belgium' au mois de 2020-03-01 00:00:00 [Recovered]: 1696 ≠ 1 + 1755 (tolérance ±50)
⚠️ Incohérence pour 'Benin' au mois de 2020-05-01 00:00:00 [Confirmed]: 232 ≠ 64 + 377 (tolérance ±50)
⚠️ Incohérence pour 'Benin' au mois de 2020-05-01 00:00:00 [Recovered]: 143 ≠ 33 + 162 (tolérance ±50)
⚠️ Incohérence pour 'China' au mois de 2020-04-01 00:00:00 [Recovered]: 76951 ≠ 74645 + 4006 (tolérance ±50)
⚠️ Incohérence pour 'Cote d'Ivoire' au mois de 2020-06-01 00:00:00 [Recovered]: 4273 ≠ 1435 + 3460 (tolérance ±50)
⚠️ Incohérence pour 'Ecuador' au mois de 2020-05-01 00:00:00 [Confirmed]: 39098 ≠ 24934 + 17277 (tolérance ±50)
⚠️ Incohérence pour 'Finland' au mois de 2020-05-01 00:00:00 [Recovered]: 5500 ≠ 3000 + 2900 (tolérance ±50)
⚠️ Incohérence pour 'France' au